# import modules and df

In [86]:
# import relevant modules
from pathlib import Path
import pandas as pd
import re

# import and setup display width
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# read df
pd.options.mode.chained_assignment = None  # default='warn'
df = pd.read_csv('bank_statement.CSV', delimiter=';')

# initial setup of the df

In [87]:
# drop irrelevant columns
df = df.drop(columns = ['account', 'valuta_date', 'iban', 'currency'])

# drop irrelevant rows
df = df.loc[(df['amount'] != '0')]

# create category, subgroup and maingroup
df['category'] = 'income'
df.loc[df['amount'].str.contains('-', flags=re.I, regex=True, na=False)
       , 'category'] = 'expenses'
df['maingroup'] = ''
df['subgroup'] = 'assign manually'

# print df for testing
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df)

# create categories, groups and searchterms

In [88]:
# create tuple for both categories
categories = ('income', 'expenses')

# create dict with all main and sub groups for expenses
expenses_groups = {'apartment': ['rent', 'electricity', 'internet'],
                   'contracts': ['streaming', 'smartphone'],
                   'dog': ['dog stuff', 'dog food'],
                   'cost of living': ['groceries/drugstore', 'pharmacy', 'barber', 'clothing'],
                   'car': ['fuel/carwash'],
                   'insurances': ['car insurance', 'liability insurance'],
                   'account stuff': ['transfers'],
                   'savings': ['savings'],
                   'other expenses': ['presents', 'videogames', 'donations', 'other expenses'],
                   'quality time': ['activites', 'restaurant and bar']}

# create dict with all main and sub groups for income
income_groups = {'salary': ['salary'],
                 'other income': ['refunds', 'other income'],
                 'account stuff': ['transfers']}

# create dict with all expense-searchterms
search_terms_expenses = {'rent': 'rent',
                         'electricity': 'vattenfall',
                         'internet': 'ewe tel',
                         'streaming': 'netflix',
                         'smartphone': 'telekom',
#                          'dog stuff': '',
                         'dog food': 'fressnapf',
                         'groceries/drugstore': 'lidl|netto|rewe|combi|famila|aleco',
                         'pharmacy': 'apotheke',
                         'barber': 'barber',
#                          'clothing': '',
                         'fuel/carwash': 'aral|avia',
                         'car insurance': 'allianz',
                         'liability insurance': 'aok',
                         'transfers': 'transfer to',
                         'savings': 'savings',
                         'presents': 'fleur',
                         'videogames': 'steam',
                         'donations': 'betterplace',
#                          'other expenses': '',
                         'activites': 'cinestar',
                         'restaurant and bar': 'burger king|mcdonalds'}

# create dict with all income-searchterms
search_terms_income = {'salary': 'spacex|wallmart',
#                        'refunds': '',
#                        'other income': '',
                       'transfers': 'transfer from'}

# auto assignment of subgroups

In [89]:
### ofcourse the subgroups can be assigned via iban or other combinations (description & iban, ...). But in this simple example the transactions can be differntiated by discription.

# assign expenses subgroups
for i in range(len(search_terms_expenses)):
    df.loc[(df['category'].str.contains('expenses', flags=re.I, regex=True, na=False))
            & (df['description'].str.contains(list(search_terms_expenses.values())[i], flags=re.I, regex=True, na=False))
            , 'subgroup'] = list(search_terms_expenses.keys())[i]

# assign income subgroups
for i in range(len(search_terms_income)):
    df.loc[(df['category'].str.contains('income', flags=re.I, regex=True, na=False))
            & (df['description'].str.contains(list(search_terms_income.values())[i], flags=re.I, regex=True, na=False))
            , 'subgroup'] = list(search_terms_income.keys())[i]

# create new df for the edited df
df_auto_subgroups = df

# print df for testing
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    # print full df
#     display(df_auto_subgroups)
    # print 'assign manually' part of df
#     display(df_auto_subgroups[df_auto_subgroups.subgroup.isin(["assign manually"])])

# manual assignment of subgroups

In [90]:
### some subgroups can not be assigned automatically and have to be assigned manually. Otherwise the plotting later on would not be accurate.


# initial messsage for the user
print('\033[1m' + '\033[4m' + 'please assign the following entries manually:\n' + '\033[0m')

# funtion to change the subgroups that have not been auto assigned
def choose_groups():
    # iterate through every row - skip the assigned subgroups and change the unsassigned ones
    for index, row in df_auto_subgroups.iterrows():
        if df_auto_subgroups.loc[index,'subgroup'] == 'assign manually':
            # print info of the transaction
            print('\033[1m' + '\033[94m' + f'Index: {index}\n' + '\033[0m')
            print(row[0:7])
            
            category = row['category']
            
            # loop if category = expenses
            if category == 'expenses':
                # print all main and subgroups
                print('\033[1m' + '\ngroups:' + '\033[0m')
                for i in expenses_groups:
                    print('\033[1m' + ' ' + f'{i}' + '\033[0m')
                    print('  ' + f'{expenses_groups[i]}')
                    
                # let the user select the maingroup
                maingroup = str(input('select maingroup: '))
                while maingroup not in expenses_groups.keys():
                    print('\033[1m' + '\033[31m' + '\nplease enter a valid maingroup!\n' + '\033[0m')
                    maingroup = str(input('select maingroup: '))
                else:
                    # let the user select the subgroup
                    subgroup = str(input('select subgroup: '))
                    while subgroup not in expenses_groups[maingroup]:
                        print('\033[1m' + '\033[31m' + '\nplease enter a valid subgroup!\n' + '\033[0m')
                        subgroup = str(input('select subgroup: '))
                    else:
                        # assign the selected subgroup
                        df_auto_subgroups.at[index, 'subgroup'] = subgroup
                        print()
                        
            # loop if category = income
            elif category == 'income':
                # print all main and subgroups
                print('\033[1m' + '\ngroups:' + '\033[0m')
                for i in income_groups:
                    print('\033[1m' + ' ' + f'{i}' + '\033[0m')
                    print('  ' + f'{income_groups[i]}')
                    
                # let the user select the maingroup
                maingroup = str(input('select maingroup: '))
                while maingroup not in income_groups.keys():
                    print('\033[1m' + '\033[31m' + '\nplease enter a valid maingroup!\n' + '\033[0m')
                    maingroup = str(input('maingroup waehlen: '))
                else:
                    # let the user select the subgroup
                    subgroup = str(input('select subgroup: '))
                    while subgroup not in income_groups[maingroup]:
                        print('\033[1m' + '\033[31m' + '\nplease enter a valid subgroup!\n' + '\033[0m')
                        subgroup = str(input('subgroup wählen: '))
                    else:
                        # assign the selected subgroup
                        df_auto_subgroups.at[index, 'subgroup'] = subgroup
                        print()
                        
            else:
                print('error')
                
        else:
            pass
        
# call manual subgroup assignment function
choose_groups()

# create new df for the edited df
df_with_subgroups = df_auto_subgroups

# print df for testing
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_with_subgroups)

Please assign the following entries manually:

Index: 13

booking_date         07.07.2023
description      ARO Restaurant
amount                   -38,99
category               expenses
maingroup                      
subgroup        assign manually
Name: 13, dtype: object

Groups:
 apartment
  ['rent', 'electricity', 'internet']
 contracts
  ['streaming', 'smartphone']
 dog
  ['dog stuff', 'dog food']
 groceries and more
  ['groceries/drugstore', 'pharmacy', 'barber', 'clothing']
 car
  ['fuel/carwash']
 insurances
  ['car insurance', 'liability insurance']
 account stuff
  ['transfers']
 savings
  ['savings']
 other expenses
  ['presents', 'videogames', 'donations', 'other expenses']
 quality time
  ['activites', 'restaurant and bar']
Select maingroup: quality time
Select subgroup: restaurant and bar

Index: 14

booking_date                    08.07.2023
description     Amazon Order 123 (dog bed)
amount                              -98,99
category                          expenses
ma

# possibility for adding cash payments

In [91]:
# create new df to edit it
df_with_cashpayment = df_with_subgroups

while True:
    cash_payment = str(input('do you want to add cash payments (y/n)? '))
    if cash_payment == 'y':
        # get user input on date and amount of the cash transaction
        date = str(input('booking_date (dd.mm.yyyy): '))
        amount = str(input('amount (incl. negative sign if it is an expense!): '))
        
        # get user input on category, maingroup and subgroup of the cash transaction
        print('\033[1m' + f"\ncategories: {categories}\n" + '\033[0m')
        category = str(input('select category: '))
        while category not in categories:
            print('\033[1m' + '\033[31m' + "\nplease enter a valid category!\n" + '\033[0m')
            category = str(input('select category: '))
        else:
            # loop to get desired expense subgroup
            if category == 'expenses':
                print('\033[1m' + '\ngroups:' + '\033[0m')
                for i in expenses_groups:
                    print('\033[1m' + ' ' + f'{i}' + '\033[0m')
                    print('  ' + f'{expenses_groups[i]}')
                maingroup = str(input('select maingroup: '))
                while maingroup not in expenses_groups.keys():
                    print('\033[1m' + '\033[31m' + '\nplease enter a valid maingroup!\n' + '\033[0m')
                    maingroup = str(input('select maingroup: '))
                else:
                    subgroup = str(input('select subgroup: '))
                    while subgroup not in expenses_groups[maingroup]:
                        print('\033[1m' + '\033[31m' + '\nplease enter a valid subgroup!\n' + '\033[0m')
                        subgroup = str(input('select subgroup: '))
                    else:
                        group = subgroup
                        
            # loop to get desired income subgroup
            elif category == 'income':
                print('\033[1m' + '\ngroups:' + '\033[0m')
                for i in income_groups:
                    print('\033[1m' + ' ' + f'{i}' + '\033[0m')
                    print('  ' + f'{income_groups[i]}')
                maingroup = str(input('select maingroup: '))
                while maingroup not in income_groups.keys():
                    print('\033[1m' + '\033[31m' + '\nplease enter a valid maingroup!\n' + '\033[0m')
                    maingroup = str(input('select maingroup: '))
                else:
                    subgroup = str(input('select subgroup: '))
                    while subgroup not in income_groups[maingroup]:
                        print('\033[1m' + '\033[31m' + '\nplease enter a valid subgroup!\n' + '\033[0m')
                        subgroup = str(input('select subgroup: '))
                    else:
                        group = subgroup
                
        # create new row with the given input (date, amount and subgroup) and attach in to the df
        new_row = pd.DataFrame({'booking_date' : date, 'amount' : amount, 'subgroup' : group, 'description' : 'cash payment'}, index=[0])
        df_with_cashpayment = pd.concat([new_row, df_with_cashpayment.loc[:]]).reset_index(drop=True)
        
    elif cash_payment == 'n':
        break
        
    else:
        print('\033[1m' + '\033[31m' + '\nplease enter y/n!\n' + '\033[0m')
        
# create category for cash transactions
df_with_cashpayment.loc[df_with_cashpayment['amount'].str.contains('-', flags=re.I, regex=True, na=False)
                       , 'category'] = 'expenses'
df_with_cashpayment.loc[~df_with_cashpayment['amount'].str.contains('-', flags=re.I, regex=True, na=False)
                       , 'category'] = 'income'
        
# print df for testing
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_with_cashpayment)

Do you want to add cash payments (y/n)? n


# auto assignment of maingroups

In [93]:
# assign maingroup for expenses
for group in expenses_groups.keys():
    values = '|'.join(expenses_groups[group])
    df_with_cashpayment.loc[(df_with_cashpayment['category'].str.contains('expenses', flags=re.I, regex=True, na=False))
                            & (df_with_cashpayment['subgroup'].str.contains(values, flags=re.I, regex=True, na=False))
                            , 'maingroup'] = group

# assign maingroup for income
for group in income_groups.keys():
    values = '|'.join(income_groups[group])
    df_with_cashpayment.loc[(df_with_cashpayment['category'].str.contains('income', flags=re.I, regex=True, na=False))
                            & (df_with_cashpayment['subgroup'].str.contains(values, flags=re.I, regex=True, na=False))
                            , 'maingroup'] = group
    
# create new df for the edited df
df_with_maingroups = df_with_cashpayment

# print df for testing
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_with_maingroups)

# final df check

In [94]:
# initial messsage for the user
print('\033[1m' + '\033[4m' + 'Please test the following spreadsheet...' + '\033[0m')
print('Especially check category, subgroup and maingroup for integrity.\n' + '\033[1m' + '\033[31m' + 'Transfers will be ignored when plotting!' + '\033[0m')

# define order of the columns
df_with_maingroups = df_with_maingroups[['booking_date', 'description', 'amount', 'category', 'maingroup', 'subgroup']]

# replace ',' with '.'
# df_with_maingroups = df_with_maingroups.apply(lambda x: x.str.replace('.','-'))
df_with_maingroups['amount'] = df_with_maingroups['amount'].replace(',','.', regex=True)

# convert column containing the transaction amount from str to float
convert_dict = {'amount': float}
df_with_maingroups = df_with_maingroups.astype(convert_dict)

# sort df by booking date and reset index
df_with_maingroups = df_with_maingroups.sort_values(by=['booking_date'])
df_with_maingroups = df_with_maingroups.reset_index(drop=True)

# print df
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_with_maingroups)

Please test the following spreadsheet...
Especially check category, subgroup and maingroup for integrity.
Transfers will be ignored when plotting!


,booking_date,description,amount,category,maingroup,subgroup
0,01.07.2023,Betterplace,-25.00,expenses,other expenses,donations
1,01.07.2023,Savings,-150.00,expenses,savings,savings
2,01.07.2023,Telekom,-8.99,expenses,contracts,smartphone
3,01.07.2023,Allianz,-80.00,expenses,insurances,car insurance
4,01.07.2023,AOK,-45.99,expenses,insurances,liability insurance
5,01.07.2023,Vattenfall,-120.00,expenses,apartment,electricity
6,01.07.2023,Netflix,-17.99,expenses,contracts,streaming
7,01.07.2023,Wallmart,1300.00,income,salary,salary
8,01.07.2023,SpaceX,1800.00,income,salary,salary
9,01.07.2023,Rent,-1200.00,expenses,apartment,rent


# possibility for manual changes

In [95]:
# create new df to edited it
df_processed = df_with_maingroups

# while loop to make changes to the df via user input
while True:
    changes = str(input('do you want to make changes (y/n)? '))
    if changes == 'y':
        # get user input (row, column and new cell content)
        row = int(input('index: '))
        col = str(input('dolumn: '))
        change = str(input('updated content (pay attention to format!): '))
        
        # make change at given location with new cell content
        df_processed.at[row, col] = change
    else:
        break

# saving the processed df ...
# ... as csf for further plotting and ...
df_processed.to_csv('bank_statement_processed.CSV', index=False, sep=';')
# ... as excel for manual inspectation
filepath_df_processed_excel = Path('results/income_and_expenses_sheet.xlsx')
filepath_df_processed_excel.parent.mkdir(parents=True, exist_ok=True)
df_processed.to_excel(filepath_df_processed_excel)

Do you want to make changes (y/n)? n
